# Research Excellence Framework results from 

We read and convert the HEFCE Research Excellence Framework (REF) data which is available from HEFCE (weird link: https://results.ref.ac.uk/(S(hlvnuqzwkag44jp3df3d4q14))/)

This is another university page with learning provider (`ukprn`) codes so the process that we will use to geocode the institutions is similar to what we did before.


## Preamble

In [ ]:
%run ../notebook_preamble.ipy

In [ ]:
import csv
import zipfile
import io

In [ ]:
from ast import literal_eval

In [ ]:
import seaborn as sn
from nuts_finder import NutsFinder

### Functions

#### Simple utilities

In [ ]:
def tidy_cols(my_csv):
    '''
    Tidies column names ie lower and replace spaces with underscores
    
    '''
    
    return([re.sub(' ','_',col.lower()) for col in my_csv.columns])

#### Create NUTS aggregations

In [ ]:
def make_nuts_estimate(data,nuts_lookup,counter,name):
    '''
    This function takes hesa data and creates a nuts estimate
    
    Args:
        data (df) where we have already selected variables of interest eg mode of employment
        nuts (dict) is the ukprn - nuts name and code lookup
        counter (str) is the variable with counts that we are interested in
    
    '''
    
    d = data.copy()
    
    #Add the nuts names and codes
    d['nuts_name'],d['nuts_code'] = [[nuts_lookup[ukprn][var] if ukprn in nuts_lookup.keys() else np.nan for ukprn in data['ukprn']] for
                                     var in ['nuts_name','nuts_code']]
    
    #We are focusing on numbers
    d[counter] = d[counter].astype(float)
    
    out = d.groupby(['nuts_name','nuts_code'])[counter].sum()
    
    out.name = name
    
    return(out)

In [ ]:
def multiple_nuts_estimates(data,nuts_lookup,variables,select_var,value):
    '''
    Creates NUTS estimates for multiple variables.
    
    Args:
        data (df) is the filtered dataframe
        select_var (str) is the variable we want to use to select values
        nuts_lookup (dict) is the lookup between universities and nuts
        variables (list) is the list of variables for which we want to generate the analysis
        value (str) is the field that contains the numerical value we want to aggregate in the dataframe
    
    '''
    
    concat = pd.concat([make_nuts_estimate(data.loc[data[select_var]==m],nuts_lookup,value,m) for m in 
              variables],axis=1)
    
    return(concat)
        
    

#### Directories etc

In [ ]:
# Create a hesa directory in raw and processed

In [ ]:
if 'ref' not in os.listdir('../../data/raw'):
    os.mkdir('../../data/raw/hef')
    
if 'ref' not in os.listdir('../../data/processed'):
    os.mkdir('../../data/processed/ref')

## 1 Load data

### University metadata

In [ ]:
#Read and evaluate the university NUTS dictionary
with open('../../data/metadata/uni_nuts.txt','r') as infile:
    uni_nuts = literal_eval(infile.read())

### HEFCE data

In [ ]:
#Read data
ref = pd.read_excel('https://results.ref.ac.uk/(S(hlvnuqzwkag44jp3df3d4q14))/DownloadFile/AllResults/xlsx',skiprows=7,na_values='-')

In [ ]:
ref.head()

In [ ]:
ref.columns = tidy_cols(ref)

In [ ]:
focus_vars = ['institution_code_(ukprn)','institution_name','unit_of_assessment_name','profile','fte_category_a_staff_submitted','4*','3*','2*','1*','unclassified']



In [ ]:
ref_2 = ref[focus_vars]

## 2. Processing

We want to do the following: 

* Estimate FTE in each category (multiply submitted by percentages)
* Create NUTS aggregates
* Save

**FTE in category**

In [ ]:
#Create the full time estimate equivalents in each category

for x in ['4*','3*','2*','1*','unclassified']:
    
    ref_2[x+'_fte'] = [fte*star/100 for fte,star in zip(ref_2['fte_category_a_staff_submitted'],ref_2[x])]

In [ ]:
#Focus on the overall variable rather than its components
ref_3 = ref_2.loc[ref['profile']=='Overall']

In [ ]:
#Now we melt the ref df so it is easier to create the aggregations later.
focus_vars_2 = ['institution_code_(ukprn)','institution_name','unit_of_assessment_name','4*_fte','3*_fte','2*_fte','1*_fte','unclassified_fte']


ref_long = ref_3[focus_vars_2].melt(id_vars=['institution_code_(ukprn)','institution_name','unit_of_assessment_name'])

In [ ]:
ref_long.head()

In [ ]:
#We rename the variable with the ukprn code so it works with our functions
ref_long.rename(columns={'institution_code_(ukprn)':'ukprn'},inplace=True)

#We reduce the institute of zoology, which does not have a UKPRN

ref_long = ref_long.loc[ref_long['ukprn']!='ZZZZZZZZ']

In [ ]:
ref_long['ukprn'] = ref_long['ukprn'].astype('float')

**Convert to NUTS**

We will subset by discicpline and aggregate over ftes


In [ ]:
out = []

#For each unique discipline
for disc in set(ref_long['unit_of_assessment_name']):
    
    #Subset by that discipline
    df_in_unit = ref_long.loc[ref_long['unit_of_assessment_name']==disc]
    
    #Aggregate over nuts
    nuts_in_unit = multiple_nuts_estimates(df_in_unit,uni_nuts,set(df_in_unit['variable']),'variable','value')
    
    #Add the discipline (unit of assessment) name so we know what everything is when we concatenate
    nuts_in_unit['unit_of_assessment_name'] = disc
    
    #Put in the list
    out.append(nuts_in_unit)

In [ ]:
#Concatenate
nuts_ref_ftes = pd.concat(out,axis=0)

**Tidy up variables**

In [ ]:
#FTE variables ordered
fte_vars = ['4*_fte','3*_fte','2*_fte','1*_fte','unclassified_fte']

nuts_ref_ftes = nuts_ref_ftes[['unit_of_assessment_name']+fte_vars]

In [ ]:
nuts_ref_ftes['total_fte'] = nuts_ref_ftes[fte_vars].sum(axis=1)

In [ ]:
#Mini exploration

high_score_discipline = nuts_ref_ftes.pivot_table(index='nuts_name',columns='unit_of_assessment_name',values='4*_fte').fillna(0)

sn.clustermap(high_score_discipline.corr())

The above suggests that there is a 'classics' cluster with traditional disciplines, a more applied cluster, and a cluster of newer perhaps less academic disciplines. It would be very interesting to dig into this much deeper.

**Save**

In [ ]:
nuts_ref_ftes.to_csv(f'../../data/processed/ref/{today_str}_ref_nuts.csv')